In [54]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [55]:
class SVM_classifier():
    def __init__(self, learning_rate, no_of_iterations, lambda_parameter):  # Initialize hyperparameters
        self.learning_rate = learning_rate
        self.no_of_iterations = no_of_iterations
        self.lambda_parameter = lambda_parameter

    def fit(self, X, Y):
        self.m, self.n = X.shape
        self.w = np.zeros(self.n)
        self.b = 0
        self.X = X
        self.Y = Y

        for _ in range(self.no_of_iterations):
            self.update_weights()

    def update_weights(self):
        y_label = np.where(self.Y <= 0, -1, 1)  # Convert Y labels: SVM works with {-1, 1} instead of {0, 1}

        for index, x_i in enumerate(self.X):
            # Decision boundary for hyperplane
            condition = y_label[index] * (np.dot(x_i, self.w) + self.b) >= 1

            # Using the Hinge Loss function
            if condition:  # Correct classification
                dw = 2 * self.lambda_parameter * self.w
                db = 0
            else:  # Misclassification
                dw = 2 * self.lambda_parameter * self.w - np.dot(x_i, y_label[index])
                db = -y_label[index]

            # Update weights and bias using gradient descent
            self.w = self.w - self.learning_rate * dw
            self.b = self.b - self.learning_rate * db

    def predict(self, X):
        output = np.dot(X, self.w) - self.b
        predicted_labels = np.sign(output)  # Assign +1 or -1 based on the sign

        print("Predicted labels shape:", predicted_labels.shape)

        y_hat = np.where(predicted_labels <= -1, 0, 1)  # Convert -1 back to 0

        return y_hat


In [56]:
model = SVM_classifier(learning_rate=0.001, no_of_iterations=1000, lambda_parameter=0.01)

In [57]:
diabetes_data = pd.read_csv('/content/diabetes.csv')

In [58]:
diabetes_data.head()

,Index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Outcome.1
0,0.0,6,148,72,35,0,33.6,0.627,50,1,1.0
1,1.0,1,85,66,29,0,26.6,0.351,31,0,-1.0
2,2.0,8,183,64,0,0,23.3,0.672,32,1,1.0
3,3.0,1,89,66,23,94,28.1,0.167,21,0,-1.0
4,4.0,0,137,40,35,168,43.1,2.288,33,1,1.0


In [59]:
diabetes_data.shape

(768, 11)

In [60]:
diabetes_data.describe()

,Index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Outcome.1
count,10.00000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,27.000000
mean,4.50000,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958,0.333333
std,3.02765,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951,0.960769
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000,-1.000000
25%,2.25000,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000,-1.000000
50%,4.50000,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000,1.000000
75%,6.75000,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000,1.000000
max,9.00000,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000,1.000000


In [61]:
diabetes_data = diabetes_data.drop(columns='Index', axis=1)
diabetes_data = diabetes_data.drop(columns='Outcome.1', axis=1)

In [62]:
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [63]:
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [64]:
diabetes_data['Outcome'].value_counts()

,count
Outcome,
0,500
1,268


In [65]:
features = diabetes_data.drop(columns='Outcome', axis=1)  # Select all columns except 'Outcome'

target = diabetes_data['Outcome']  # Select only the 'Outcome' column


In [66]:
print(features)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [67]:
print(target)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


In [68]:
scaler = StandardScaler()

In [69]:
scaler.fit(features)

StandardScaler()

In [70]:
standardized_data = scaler.transform(features)

In [71]:
print(standardized_data)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]


In [72]:
features = standardized_data
target = diabetes_data['Outcome']

In [73]:
print(features)
print(target)

[[ 0.63994726  0.84832379  0.14964075 ...  0.20401277  0.46849198
   1.4259954 ]
 [-0.84488505 -1.12339636 -0.16054575 ... -0.68442195 -0.36506078
  -0.19067191]
 [ 1.23388019  1.94372388 -0.26394125 ... -1.10325546  0.60439732
  -0.10558415]
 ...
 [ 0.3429808   0.00330087  0.14964075 ... -0.73518964 -0.68519336
  -0.27575966]
 [-0.84488505  0.1597866  -0.47073225 ... -0.24020459 -0.37110101
   1.17073215]
 [-0.84488505 -0.8730192   0.04624525 ... -0.20212881 -0.47378505
  -0.87137393]]
0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


In [74]:
X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.2, random_state=2)

In [75]:
print("Values of the X =",X_train.shape,X_test.shape)

Values of the X = (614, 8) (154, 8)


In [76]:
print("Values of the Y =",Y_train.shape,Y_test.shape)

Values of the Y = (614,) (154,)


In [77]:
classifier = SVM_classifier(learning_rate=0.001, no_of_iterations=1000, lambda_parameter=0.01)

In [78]:
classifier.fit(X_train, Y_train)
print("X shape:", X_train.shape)
print("y shape:", Y_train.shape)


X shape: (614, 8)
y shape: (614,)


In [79]:
X_train_prediction = classifier.predict(X_train)
print("X_train_prediction =",X_train_prediction.shape)


training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print('Accuracy score on training data = ', training_data_accuracy)


Predicted labels shape: (614,)
X_train_prediction = (614,)
Accuracy score on training data =  0.6384364820846905


In [80]:
X_test_prediction = classifier.predict(X_test)
print("X_test_prediction =",X_test_prediction.shape)

test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print('Accuracy score on test data = ', test_data_accuracy)

Predicted labels shape: (154,)
X_test_prediction = (154,)
Accuracy score on test data =  0.5584415584415584
